In [2]:
import torch.nn as nn
import numpy as np

# Linear regression
Doing the same thing as linear_regression but with pytorch

In [4]:
inputs = np.array([[73,76,43],[91,88,64],[87,134,58],
                   [102,43,37],[69,96,70],[73,67,43],
                   [91,88,64],[87,134,58],[102,43,37],
                   [69,96,70],[73,67,43],[91,88,64],
                   [87,134,58],[102,43,37],[69,96,70]],
                 dtype='float32')

targets = np.array([[56,70], [81,101], [119,133],
                   [22,37], [103,119], [56,70],
                   [81,101], [119,133], [22,37],
                   [103,119], [56,70], [81,101],
                   [119,133], [22,37], [103,119]],
                  dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

# Process in batches
Import TensorDataset for easier batch processing

In [5]:
from torch.utils.data import TensorDataset

In [7]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  76.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [8]:
# Above we are accessing slices of data very easily

In [9]:
from torch.utils.data import DataLoader

In [10]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [11]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.]])
tensor([[119., 133.],
        [ 81., 101.],
        [ 22.,  37.],
        [ 56.,  70.],
        [103., 119.]])


# nn.Linear
We now define the model with nn.Linear class from pytorch

In [13]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5176, -0.1685,  0.1554],
        [ 0.2391,  0.3398,  0.5437]], requires_grad=True)
Parameter containing:
tensor([ 0.5571, -0.3718], requires_grad=True)


In [15]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.5176, -0.1685,  0.1554],
         [ 0.2391,  0.3398,  0.5437]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5571, -0.3718], requires_grad=True)]

In [16]:
# Generate predictions
preds = model(inputs)
preds

tensor([[32.2145, 66.2826],
        [42.7719, 86.0810],
        [32.0170, 97.4927],
        [51.8536, 58.7399],
        [30.9691, 86.8024],
        [33.7313, 63.2246],
        [42.7719, 86.0810],
        [32.0170, 97.4927],
        [51.8536, 58.7399],
        [30.9691, 86.8024],
        [33.7313, 63.2246],
        [42.7719, 86.0810],
        [32.0170, 97.4927],
        [51.8536, 58.7399],
        [30.9691, 86.8024]], grad_fn=<AddmmBackward>)

# Loss Function
built in loss function mse_loss

In [17]:
# Import nn.functional
import torch.nn.functional as F

In [18]:
# This contains many useful loss functions

In [19]:
# Define the loss function
loss_fn = F.mse_loss

In [20]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(1865.4161, grad_fn=<MseLossBackward>)


# Optimizer
`optim.SGD` which stands for `stchastic gradient descent`

In [21]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

# Train the model
We are now ready to train the model. Exact same process:
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

Here we will work with batches of data, instead of processing the entire training data in every iteration.
We will define a utility function `fit` which trains the model for a given number of epochs.

In [27]:
def fit(num_epochs, model, loss_fn, opt):
    # Repeat for a given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate Loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute the gradients
            loss.backward()
            
            # 4. Update parameters using gradient
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))


In [28]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 318.9207
Epoch [20/100], Loss: 251.7028
Epoch [30/100], Loss: 302.6771
Epoch [40/100], Loss: 136.5068
Epoch [50/100], Loss: 22.6537
Epoch [60/100], Loss: 55.4797
Epoch [70/100], Loss: 65.5131
Epoch [80/100], Loss: 28.7063
Epoch [90/100], Loss: 38.8650
Epoch [100/100], Loss: 16.4532


In [29]:
# Let's generate preds and see how far off we are
preds = model(inputs)
preds

tensor([[ 65.8330,  78.0841],
        [ 81.2584,  99.4413],
        [117.0224, 133.1465],
        [ 29.3200,  41.5000],
        [ 95.6971, 114.4410],
        [ 58.4703,  70.7961],
        [ 81.2584,  99.4413],
        [117.0224, 133.1465],
        [ 29.3200,  41.5000],
        [ 95.6971, 114.4410],
        [ 58.4703,  70.7961],
        [ 81.2584,  99.4413],
        [117.0224, 133.1465],
        [ 29.3200,  41.5000],
        [ 95.6971, 114.4410]], grad_fn=<AddmmBackward>)

In [30]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])